In [1]:
%load_ext autoreload
%autoreload 2

from dataset import DialogLMDataset
from torch.utils.data import DataLoader
from collate_fns import DialogCollate, DialogCollateExperimental
from models import NegativePositiveTrainingLM
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler


import torch
from transformers import AutoConfig, AutoTokenizer
from sklearn.model_selection import train_test_split

# utils
from data_utils import InfiniteDataGen
from logging_utils import create_json, update_json, load_json

import os
import json
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)
import datetime
import random

In [3]:
df = pd.read_csv('eval_set_10k.csv', index_col=0)
df

,id,context,response
0,"t3_eebfd,t1_c17hmnx,t1_c17i1m0",title : Opinions on the M14 and G3 optics ? EOS I am glad they are still balanced compared to the rest of the weapons .,"Yes , I was nervous too , but I think everyone's fears are allayed ."
1,"t3_eebfd,t1_c17hqbv,t1_c17hw7t",Wake Island,All three were amazing maps
3,"t3_eebfd,t1_c17j7rv,t1_c17javx","title : Opinions on the M14 and G3 optics ? EOS it felt floaty and inaccurate , and I had trouble hitting targets That's because the ACOG fuckin sucks , it's always floaty and inaccurate","The ACOG is fantastic on most weapons . I can demolish people at sniper range with an SMG with it . Just not with the M14 , for some reason ."
4,"t3_eebfd,t1_c17lws5,t1_c17lzfb",Wake Island EOS Strike at Sharqand Peninsula,Mashtuur City
5,"t3_eebfd,t3_eebfd,t1_c17hmnx",title : Opinions on the M14 and G3 optics ?,I am glad they are still balanced compared to the rest of the weapons .
...,...,...,...
21836,"t3_eei3e,t1_c17il3e,t1_c17jmgm",Obviously they do not or all hell would break loose . It is in the constitution ffs .,And yet creationism exists in schools . Go figure .
21837,"t3_eei3e,t1_c17ildw,t1_c17iqmn","title : Good answer kid . EOS I guess it shouldn't surprise me that even a kid could run the Catholic Church better than the Pope . EOS an atheist agnostic kid , that is . EOS ' Dad , this bible shit is boring and they can't even use the magics in an entertaining manner . I will stick with my Harry Potter series ! ' EOS Church is like a book of the week club that never rotates its reading list .",And the majority of the members of that book club never actually read the book .
21840,"t3_eei3e,t1_c17im4k,t1_c17inn4",Wi n t trei a h liday in Sweden this yer ?,See the l veli lakes ...
21842,"t3_eei3e,t1_c17im70,t1_c17inar",title : Good answer kid . EOS Amazing . How old is this kid ?,thanks i taught him well . he will be 16 in january


In [5]:
# tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    'microsoft/DialoGPT-small'
)

# train sets
dataset_eval = DialogLMDataset(
    df = df, #.iloc[:10],
    src_col = 'context',
    src_eos_token = ' EOS ',
    tokenizer = tokenizer,
    tgt_col = 'response',  # response
)

print()
print(dataset_eval[3])
tokenizer.decode(dataset_eval[-1]['src'])

Processing row: 9000 of 10000
{'src': [54, 539, 5451, 50256, 31584, 379, 15465, 80, 392, 24078], 'tgt': [44, 1077, 28047, 333, 2254]}


"Good answer kid.<|endoftext|>I guess it shouldn't surprise me that even a kid could run the Catholic Church better than the Pope.<|endoftext|>an atheist agnostic kid, that is.<|endoftext|>at least he would be able to reason<|endoftext|>Let's let them marry men? And Preferably men their own age bracket,<|endoftext|>Homosexuality and pedophilia ( even of young boys ) are not linked.<|endoftext|>Sure they are. They are both sexual orientations that environmentally develop since childhood."

In [7]:
# example dataset and dataloader usage

# collate object
collate_fn = DialogCollateExperimental(
    tokenizer = tokenizer,
    max_len = 512,
    _targets_ignore_index = -100,
    _pad_token_id = tokenizer.eos_token_id  # 0
)

# dataloader
loader_eval = DataLoader(
    dataset_eval,
    batch_size = 2,
    shuffle = False,
    collate_fn = collate_fn
)

# example usage
for batch in loader_eval:
    break

example_id = 0
print('input_ids:\n',       batch['input_ids'][example_id])
# print('position_ids:\n',    batch['position_ids'][example_id])
# print('token_type_ids:\n',  batch['token_type_ids'][example_id])
print('target_ids:\n',      batch['target_ids'][example_id])
# print('attention_masks:\n', batch['attention_masks'][example_id])

tokenizer.decode(batch['input_ids'][example_id])

input_ids:
 tensor([   46, 11635,   507,   319,   262,   337,  1415,   290,   402,    18,
        36237,  5633, 50256,    40,   716,  9675,   484,   389,   991, 12974,
         3688,   284,   262,  1334,   286,   262,  3777,   764, 50256,  5297,
          837,   314,   373, 10927,  1165,   837,   475,   314,   892,  2506,
          338, 10251,   389,   477, 16548,   764, 50256])
target_ids:
 tensor([   46, 11635,   507,   319,   262,   337,  1415,   290,   402,    18,
        36237,  5633, 50256,    40,   716,  9675,   484,   389,   991, 12974,
         3688,   284,   262,  1334,   286,   262,  3777,   764, 50256,  5297,
          837,   314,   373, 10927,  1165,   837,   475,   314,   892,  2506,
          338, 10251,   389,   477, 16548,   764, 50256])


"Opinions on the M14 and G3 optics?<|endoftext|>I am glad they are still balanced compared to the rest of the weapons.<|endoftext|>Yes, I was nervous too, but I think everyone's fears are allayed.<|endoftext|>"

Explanation of target IDs: `-100` indicates that this position will not contribute to the loss. This behavior is specified by setting the `ignore_index` option in the `CrossEntropy` loss function, which by default takes the value `-100`. In order to decode the targets, we replace `-100`'s arbitrarily with `0`'s, and the ID `0` happens to correspond to the exclamation mark.

In [13]:
# which experiment to load?
EXPERIMENT_TO_LOAD = 'baseline_censored_reddit_data'
CHECKPOINT_TO_LOAD = 'epoch-2_trainloss-1.804484_valloss-2.243514'
LOGGING_DIR        = './logs/'
MODEL_SAVE_DIR     = './checkpoints/'

# load config
log_dict = load_json(
    target_dir = os.path.join(LOGGING_DIR, EXPERIMENT_TO_LOAD),
    filename   = CHECKPOINT_TO_LOAD + '.json'
)

# grab the config from the logs
CONFIG = log_dict['config']

# model
model_config = AutoConfig.from_pretrained(CONFIG['HUGGINGFACE_MODEL_NAME'])
gradient_accumulation_steps = 1
model_wrapper = NegativePositiveTrainingLM(
    model_name = CONFIG['HUGGINGFACE_MODEL_NAME'],
    model_config = model_config,
    pos_lr = CONFIG['POS_LR'], 
    neg_lr = CONFIG['NEG_LR'],
    num_opt_steps = len(loader_eval) // CONFIG['GRAD_ACCUM_STEPS'] * CONFIG['TRAIN_ITERS'],
    tokenizer = tokenizer,
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
)

# load
model_wrapper.load_model(
    checkpoint_root = CONFIG['MODEL_SAVE_DIR'],
    experiment_name = CONFIG['EXPERIMENT_NAME'],
    checkpoint_name = CHECKPOINT_TO_LOAD,
)

In [14]:
# data loaders
eval_sampler = SequentialSampler(dataset_eval)
loader_eval = DataLoader(
    dataset_eval,
    sampler = eval_sampler,
    batch_size = 2,
    # shuffle = False,
    collate_fn = collate_fn,
    drop_last = True
)

# reproducibility
random.seed(CONFIG['SEED'])
np.random.seed(CONFIG['SEED'])
torch.manual_seed(CONFIG['SEED'])
torch.cuda.manual_seed_all(CONFIG['SEED'])
eval_loss   = model_wrapper.perplexity_on_dataset(loader_eval)
print('\nVal perplexity:', eval_loss)

Evaluating batch: 4990 of 5000
Val perplexity: tensor(140.2176)
